In [1]:
import numpy as np
import h5py

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, SimpleRNN, GRU, TimeDistributed

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [54]:
from preprocess import text_to_words

# Generate a list of words (including newline)
corpus = "english"
textfile = "wordlists/" + corpus + ".txt"
words = text_to_words(textfile)

print(words[0:5])

['telescope\n', 'finish\n', 'tree\n', 'tremble\n', 'about\n']


In [55]:
# Generate the set of unique characters (including newline)
# https://stackoverflow.com/questions/952914/making-a-flat-list-out-of-list-of-lists-in-python
chars = sorted(list(set([char for word in words for char in word])))

VOCAB_SIZE = len(chars)
N_WORDS = len(words)
MAX_WORD_LEN = 12  # maximum company name length


print(N_WORDS, "words\n")
print("vocabulary of", len(chars), "characters, including the \\n:")
print(chars)
print("\nFirst two sample words:")
print(words[0:2])

2577 words

vocabulary of 28 characters, including the \n:
['\n', '\x1a', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

First two sample words:
['telescope\n', 'finish\n']


In [56]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

In [57]:
X = np.zeros((N_WORDS, MAX_WORD_LEN, VOCAB_SIZE))
Y = np.zeros((N_WORDS, MAX_WORD_LEN, VOCAB_SIZE))

for word_i in range(N_WORDS):
    word = words[word_i]
    chars = list(word)
    word_len = len(word)
    
    for char_j in range(min(len(word), MAX_WORD_LEN)):
        char = chars[char_j]
        char_ix = char_to_ix[char]
        X[word_i, char_j, char_ix] = 1
        if char_j > 0:
            Y[word_i, char_j - 1, char_ix] = 1  # the 'next char' at time point char_j

In [58]:
LAYER_NUM = 1
HIDDEN_DIM = 50

model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
#model.add(SimpleRNN(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
#for i in range(LAYER_NUM - 1):
#    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, None, 50)          15800     
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 28)          1428      
_________________________________________________________________
activation_5 (Activation)    (None, None, 28)          0         
Total params: 17,228
Trainable params: 17,228
Non-trainable params: 0
_________________________________________________________________


In [60]:
def temp_scale(probs, temperature = 1.0):
    # a low temperature (< 1 and approaching 0) results in the char sampling approaching the argmax.
    # a high temperature (> 1, approaching infinity) results in sampling from a uniform distribution)
    probs = np.exp(np.log(probs) / temperature)
    probs = probs / np.sum(probs)
    return probs
    
    
def generate_word(model, temperature = 1.0, min_word_length = 4):
    X = np.zeros((1, MAX_WORD_LEN, VOCAB_SIZE))
    
    # sample the first character
    initial_char_distribution = temp_scale(model.predict(X[:, 0:1, :]).flatten(), temperature)
    
    ix = 0
    while ix == 0:  # make sure the initial character is not a newline (i.e. index 0)
        ix = int(np.random.choice(VOCAB_SIZE, size = 1, p = initial_char_distribution))
    
    X[0, 0, ix] = 1
    generated_word = [ix_to_char[ix].upper()]  # start with first character, then later successively append chars
    
    # sample all remaining characters
    for i in range(1, MAX_WORD_LEN):
        next_char_distribution = temp_scale(model.predict(X[:, 0:i, :])[:, i-1, :].flatten(), temperature)

        
        ix_choice = np.random.choice(VOCAB_SIZE, size = 1, p = next_char_distribution)
        # ix_choice = np.argmax(next_char_distribution)  # <- corresponds to sampling with a very low temperature
        ctr = 0
        while ix_choice == 0 and i < min_word_length:
            ctr += 1
            # sample again if you picked the end-of-word token too early
            ix_choice = np.random.choice(VOCAB_SIZE, size = 1, p = next_char_distribution)
            if ctr > 1000:
                print("caught in a near-infinite loop. You might have picked too low a temperature and the sampler just keeps sampling \\n's")
                break
            
        
        next_ix = int(ix_choice)
        X[0, i, next_ix] = 1
        if next_ix == 0:
            break
        generated_word.append(ix_to_char[next_ix])
    
    return ('').join(generated_word)

In [ ]:
NUM_EPOCHS = 500
BATCH_SIZE = 64  # or: N_WORDS

epoch = 0
while epoch < NUM_EPOCHS:
    model.fit(X, Y, batch_size=BATCH_SIZE, verbose=0, epochs=1)
    epoch += 1
    
    # if epoch % 100 == 0:
    #     print("epoch", epoch)
    
    # print a word every few epochs to watch the algorithm learn
    if epoch % 50 == 0:
        print("epoch " + str(epoch) + ": " + generate_word(model))
        
model.save(corpus + "_" + str(epoch) + 'epochs.h5')

epoch 50: Unsiy
epoch 100: Laigted
epoch 150: Oidd
epoch 200: Vertait
epoch 250: Nate


In [49]:
## Load one of these models if you have trained them before and want to skip re-training
from keras.models import load_model

# model = load_model("behemoth_500epochs.h5")
# model = load_model("pokemon_500epochs.h5")
# model = load_model("german_500epochs.h5")

In [ ]:
# Print a few words with the final model:

for _ in range(30):
    print(generate_word(model, temperature = 1.2, min_word_length = 4))

### TODO

- Use a keras callback for the word generation e50epochs
- Go to codementor.io and have an expert review your code, e.g. from [this guy](https://www.codementor.io/mepstein68)